In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from utils import seleccion_txt, txt_read, get_fakes, cabeza_y_cola, corta
from ut.textmining import get_word_matrix

PATH_CALIBRE = 'c:/Users/milen/Biblioteca de calibre/'
lang = "ES"  # >>>

c:\Users\milen\AppData\Local\Programs\Python\Python38\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# 1. Summary creation

## 1.1 libros de referencia para hacer el tf-idf


In [5]:
key = 'freakonomics'  # keyword para seleccionar el libro

last, all_ = seleccion_txt(PATH_CALIBRE)  # trae todoslos título de libros en español
# get the filename with path from all_ list
file = [x for x in all_ if key in x.lower()][0]  # elejimos el primero

date_es = 20220703 if lang == 'EN' else 20200504
files_es, _ = seleccion_txt(PATH_CALIBRE, fecha=date_es)
files = [file] + files_es # usamos libros para generar el corpus para el tfidf
doc_list = [txt_read(x) for x in files] # lee los archivos y los pone en una lista

** La ultima fecha de ficheros es:  20230711
                                                     0
0             V. I. Lenin. - Materialismo y e - ().txt
1                          Mi lucha - Adolf Hitler.txt
2        El Conde de Montecristo - Alejandro Dumas.txt
3       La casa de los espiritus - Allende, Isabel.txt
4         La Araucana - Alonso de Ercilla Y Zuniga.txt
..                                                 ...
163  Heaven Is for Real_ A Little Bo - Todd Burpo; ...
164        La esfinge de los hielos - Verne, Julio.txt
165           El planeta americano - Vicente Verdu.txt
166  El hombre en busca de sentido - Viktor Frankl.txt
167                  Homo Deus - Yuval Noah Harari.txt

[168 rows x 1 columns]
                                                    0
0      La casa de los espiritus - Allende, Isabel.txt
1                  Las Mil y una Noches - Anonimo.txt
2    2001. Una odisea espacial - Arthur C. Clarke.txt
3         En las profundidades - Arthur C. Clarke.txt
4

In [ ]:
files

In [ ]:
vector_matrix, vocab, _ = get_word_matrix(doc_list)

In [ ]:
dic_fake, di_counts = get_fakes(doc_list, files, vector_matrix, vocab, lang)

In [ ]:
dic_fake = {0: dic_fake[0]}

In [ ]:
pd.DataFrame.from_dict(dic_fake)

In [ ]:
j = {dic_fake[k]['title']: dic_fake[k] for k in dic_fake}

# imagen

In [ ]:
from PIL import Image
from ipywidgets import fixed, interactive
from ut.images import crop
from utils import get_books, get_image_path, upload_lib_summary, get_book_datas
from ut.io import get_filename
from ut.base import json_read

PATH_CALIBRE = 'c:/Users/milen/Biblioteca de calibre/'

In [ ]:
image= get_image_path(file)
titulo = get_filename(file, True).split(' - ')[0]
print(titulo) #ojo que puede estar cortado por Calire si es muy largo... quizas deberíamos cogerlo de la carpeta..
im = Image.open(image)

In [ ]:
if im.size[0] > 700:
    im = im.reduce(2)
im.reduce(4)

In [ ]:
u = interactive(crop, f=(0.1, 1, 0.05),
                sx=(1, int(im.size[0] * .5)),
                sy=(1, int(im.size[1] * .9)),
                img=fixed(im))
u

In [ ]:
si = u.result.size[0]
a = 200
b = min(si, 2 * a)
im_low = u.result.resize((a, a))
im_hi = u.result.resize((b, b))


In [ ]:
base = 'data_out/_images/{}/{}.jpg'
im_low.save(base.format('low', titulo))
im_hi.save(base.format('hi', titulo))

# otherb

In [ ]:
# texto, img, titulo, d_summary = get_book_datas('Freak')

# Dividir en partes

In [ ]:
path = dic_fake[0]['path']
texto = txt_read(path)

In [ ]:
# te qudas con la i del lo que parece el final del libro
partes, df = cabeza_y_cola(texto, 20)

In [ ]:
fin = 802  # >>>
ini = 15  # >>>

# d_summary['min'], d_summary['max'] = ini, fin

In [ ]:
partes, df = corta(partes, df, ini, fin)

In [ ]:
df = df.reset_index().rename(columns={'index': 'i'})
df['ii'] = 0 # para identificar dentro de un párrafo largo que romperemos


In [ ]:
import random
from ut.base import json_read, json_save, make_folder, json_update
from utils import crea_capsulas_max, get_parrafos, get_final_parrfs, speakers_test, get_df_capitulos, \
    get_dic_capitulos, update_di_capi, procesa_capitulo, get_book_datas, SUMMARIES_JSON, sample_speaker, test_voices_en, \
    CONTENT_JSON
from ut.textmining import palabras_representativas

LIM = 850  # largo de las cápsulas, límite de lo que puede leer el sinte

final, partes = get_final_parrfs(df, LIM)
final

In [ ]:
d_capsulas = crea_capsulas_max(partes, final, lmax=LIM, verbose=False)
caps = ['.\n'.join(d_capsulas[x]['texto']) for x in d_capsulas]  # todo probar si sintetizador lee punto aparte

In [ ]:
caps[12]  # las cápsulas son las que puede leer de una sola vez

In [ ]:
df_capitulos = get_df_capitulos(caps)
df_capitulos

In [ ]:
d_capitulos = get_dic_capitulos(df_capitulos)

In [ ]:
capitulos = ['\n '.join(d_capitulos[cap]['capsulas']) for cap in d_capitulos]

In [ ]:
k = list(j.keys())[0]
d_summary = j[k]
capitulos_titles = palabras_representativas(capitulos,
                                            l_exclude=d_summary['names'],
                                            max_df=.4,  # .8  proporción de documentos. si lo bajamos quitamos los muy frecuentes
                                            min_df=.2)  # .2  % de docs. Si lo subo quito palabras poco frecuentes
capitulos_titles
update_di_capi(d_capitulos, capitulos_titles, d_summary, titulo)
path_book = make_folder('data_out/' + titulo + '/')
json_save(d_capitulos, path_book + CONTENT_JSON)

# AUDIO

In [ ]:
d_capitulos=json_read(path_book + CONTENT_JSON)

In [ ]:
import torch
from omegaconf import OmegaConf

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

models = OmegaConf.load('latest_silero_models.yml')
available_languages = list(models.tts_models.keys())

for lang in available_languages:
    modeli = list(models.tts_models.get(lang).keys())
    print(f'Available models for {lang}: {modeli}')

In [ ]:
# configuración
language = d_summary['idioma'].lower()
model_id = 'v3_es' if language == 'es' else 'v3_en'

sample_rate = 48000
put_accent = True
put_yo = True

In [ ]:
# cargamos el modelo
device = torch.device('cpu')  # or cuda, pero no me funciona

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

In [ ]:
sps = [x for x in model.speakers if x != 'random']

In [ ]:
d_capitulos

In [ ]:
# convert the keys to int
d_capitulos = {int(k): v for k, v in d_capitulos.items()}

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from utils import speakers_test

In [ ]:
if language == 'es':
    speakers_test(model,
                  txt=d_capitulos[1]['capsulas'][0][:450],
                  lan='es'
#                  txt='Millonarios por una semana.\n Cuando no se tiene una chaucha en el bolsillo, no es muy amplia la gama de actividades elegibles para matar el tiempo. Con Diego y Vittorio nos juntábamos casi todos los d'
                 )